# Estimating the aDDM
### Author: Brenden Eum (2023)

This script will read already-transformed data containing trial-by-trial details and fixations and fit the aDDM to this data. It uses the Tavares Toolbox (Tavares et al. 2017) rewritten for Julia by Lynn in Summer 2023. See https://github.com/aDDM-Toolbox/ADDM.jl for the toolbox. 

I've made a changes to the toolbox in order to suit my project. Some of these are small changes, like changing object types, saving outputs to csv files, or changing the grid for grid search. I don't document small changes since they'll probably change over multiple iterations. I do document major changes to the toolbox: (1) The toolbox now fits the model to the output of load_data_from_csv instead of simulating data; (2) The toolbox no longer throws a domain error when $\theta \notin [0,1]$.

Inputs for this notebook:
- Data
    - data/expdata*.csv:    Data pertaining to the details of each trial. An observation is a subject-trial. It must include the following columns [parcode,trial,rt,choice,item_left,item_right], which correspond to [subject, trial, response time in ms, choice (-1 left, 1 right), value of left item, value of right item] respectively.
    - data/fixations*.csv:  Data pertaining to the fixations within each trial. An observation is a subject-trial-fixation. It must include columns [parcode,trial,fix_item,fix_time], which correspond to [subject, trial, fixation location (1 left, 2 right, otherwise other), fixation duration in ms].
- Code
    - src/ addm_grid_search.jl: Takes a grid of parameters, makes every combination, and calculates negative log likelihood (NLL) of aDDM for every combo given some data.
    - addm.jl: Defines aDDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - ddm.jl: Defines DDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - util.jl: Contains functions for prepping data for aDDM fitting and estimating empirical fixation properties.


Outputs for this notebook:
- GainFit.csv:  aDDM estimates for the gain condition. Estimates are for subjects included in the input datasets.
- LossFit.csv:  aDDM estimates for the loss condition.

# Preamble

A set of standard things I like to do before getting started with any notebook.

In [1]:
import Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
using DataFrames
using CSV

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


# Activate the Environment

Typically this happens by

```
using Pkg
Pkg.activate("addm")
```

**Note that single vs. double quotes matter.**

But this does not work as expected in a notebook unless the `Project.toml` is in the same folder as the notebook file.

In [2]:
pwd()
Base.load_path()
include("../../src/addm_grid_search.jl")
Base.load_path()

  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


3-element Vector{String}:
 "/Users/brenden/Toolboxes/ADDM.jl/docs/src/addm/Project.toml"
 "/Users/brenden/.julia/environments/v1.9/Project.toml"
 "/opt/homebrew/Cellar/julia/1.9.3/share/julia/stdlib/v1.9"

# Fit the aDDM

## Set Up the Grid for Search

The plan is to use a single grid to fit the aDDM separately to both the gain and the loss data. That way, we know that it's not our choice of the grid that is driving our results since the same grid is applied to both conditions.

In [3]:
dLow = .000;
dHigh = .006;
σLow = .01;
σHigh = .10;
θLow = 0;
θHigh = 2;
n = 1;

dList = round.(LinRange(dLow, dHigh, 4), digits=4); #7
σList = round.(LinRange(σLow, σHigh, 4), digits=2); #10
θList = round.(LinRange(θLow, θHigh, 4), digits=2); #21

println(collect(dList))
println(collect(σList))
println(collect(θList))

[0.0, 0.002, 0.004, 0.006]
[0.01, 0.04, 0.07, 0.1]
[0.0, 0.67, 1.33, 2.0]


## Fit Gain Data

Lynn's version of the Tavares toolbox in Julia runs so quickly, we almost don't need to run a mulitple-iteration grid search to obtain high-resolution estimates. That being said, I do think a 2-iteration grid search will allow us to obtain very high-resolution estimates, so we'll go with that.

### Low-res Grid Search

In [4]:
dInit = 0.005; # These can be anything. They exist because you need an aDDM object.
σInit = 0.07;
θInit = 0.3;

addm = aDDM(dInit, σInit, θInit);

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)));
dGain = Vector{Float64}(undef, subjectCount)
σGain = Vector{Float64}(undef, subjectCount)
θGain = Vector{Float64}(undef, subjectCount)
ind = 1

@showprogress for subject in collect(keys(dataGain))

  dEst, σEst, θEst = aDDM_grid_search(addm, dataGain, dList, σList, θList, n, subject)

  dGain[ind] = dEst[1]
  σGain[ind] = σEst[1]
  θGain[ind] = θEst[1]
  ind += 1

end

dfGain = DataFrame(
  subject = collect(keys(dataGain)),
  d = dGain,
  s = σGain,
  t = θGain
)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:31


Row,subject,d,s,t
,String,Float64,Float64,Float64
1,215,0.002,0.04,1.33
2,229,0.004,0.04,0.67
3,217,0.002,0.04,0.67
4,204,0.004,0.04,0.67
5,220,0.004,0.04,0.67
6,219,0.002,0.07,0.67
7,216,0.004,0.07,0.67
8,211,0.002,0.04,1.33
9,209,0.002,0.04,0.67


### High-res Grid Search

In [8]:
dfGain[(dfGain.subject .== "201"), "d"][1]

0.002

In [10]:
"""
ind = 1
@showprogress for subject in collect(keys(dataGain))

    d = dfGain[(dfGain.subject .== subject), "d"][1]
    σ = dfGain[(dfGain.subject .== subject), "s"][1]
    θ = dfGain[(dfGain.subject .== subject), "t"][1]

    dLow = max(0, d-0.001);
    dHigh = d+0.001;
    σLow = max(0.001, σ-0.01);
    σHigh = σ+0.01;
    θLow = θ-0.1;
    θHigh = θ+0.1;

    dList = round.(LinRange(dLow, dHigh, 4), digits=4);
    σList = round.(LinRange(σLow, σHigh, 4), digits=3);
    θList = round.(LinRange(θLow, θHigh, 4), digits=2);  

    dEst, σEst, θEst = aDDM_grid_search(addm, dataGain, dList, σList, θList, n, subject)
  
    dGain[ind] = dEst[1]
    σGain[ind] = σEst[1]
    θGain[ind] = θEst[1]
    ind += 1
  
  end
"""

Progress: 100%|█████████████████████████████████████████| Time: 0:40:43


## Fit Loss Data

In [ ]:
dInit = 0.005;
σInit = 0.07;
θInit = 0.3;

addm = aDDM(dInit, σInit, θInit);

dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(data)));
dLoss = Vector{Float64}(undef, subjectCount)
σLoss = Vector{Float64}(undef, subjectCount)
θLoss = Vector{Float64}(undef, subjectCount)
ind = 1

@showprogress for subject in collect(keys(data))

  dEst, σEst, θEst = aDDM_grid_search(addm, dataLoss, dList, σList, θList, n, subject)

  dLoss[ind] = dEst[1]
  σLoss[ind] = σEst[1]
  θLoss[ind] = θEst[1]
  ind += 1

end

dfLoss = DataFrame(
  subject = collect(keys(data)),
  d = dLoss,
  s = σLoss,
  t = θLoss
)